<a href="https://colab.research.google.com/github/ArivoliRajan/GCP-Data-Engineer-Study-Guide/blob/master/PhotoLibrary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Titanic Data to BigQuery



## Setting up Data



In [1]:
# log in with the email account associated with your GCP.
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Installing and loading libraries

In [2]:
!pip install -q apache_beam[gcp]
import apache_beam as beam
import csv


     |████████████████████████████████| 2.9MB 2.8MB/s 
     |████████████████████████████████| 1.2MB 49.1MB/s 
     |████████████████████████████████| 153kB 61.9MB/s 
     |████████████████████████████████| 81kB 21.7MB/s 
     |████████████████████████████████| 51kB 22.4MB/s 
     |████████████████████████████████| 61kB 21.4MB/s 
     |████████████████████████████████| 102kB 32.9MB/s 
     |████████████████████████████████| 92kB 32.1MB/s 
     |████████████████████████████████| 163kB 58.0MB/s 
     |████████████████████████████████| 174kB 60.6MB/s 
     |████████████████████████████████| 112kB 54.9MB/s 
ERROR: multiprocess 0.70.8 has requirement dill>=0.3.0, but you'll have dill 0.2.9 which is incompatible.


/usr/local/lib/python3.6/dist-packages/apache_beam/__init__.py:84: UserWarning: Some syntactic constructs of Python 3 are not yet fully supported by Apache Beam.
  'Some syntactic constructs of Python 3 are not yet fully supported by '


## Data Load

In [0]:
fname = 'titanic.csv'

In [0]:
# get the file from google cloud storage.

from google.cloud import storage

project_id = 'titanicproject'
client = storage.Client(project=project_id)

bucket = client.get_bucket('arivolibucket')

blob2 = bucket.blob(fname)
blob2.download_to_filename(fname)

In [6]:
ls -al

total 88
drwxr-xr-x 1 root root  4096 Sep  8 19:29 ./
drwxr-xr-x 1 root root  4096 Sep  8 19:13 ../
-rw-r--r-- 1 root root  2497 Sep  8 19:23 adc.json
drwxr-xr-x 1 root root  4096 Sep  8 19:23 .config/
drwxr-xr-x 1 root root  4096 Aug 27 16:17 sample_data/
-rw-r--r-- 1 root root 60302 Sep  8 19:29 titanic.csv


In [7]:
import pandas as pd

df = pd.read_csv(fname)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [8]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
df[pd.isnull(df).any(axis=1)].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S


## Schema Prep

In [10]:
cols_df = pd.DataFrame(df.dtypes).reset_index()
cols_df.columns = ['Field',"Type"]
cols_df

,Field,Type
0,PassengerId,int64
1,Survived,int64
2,Pclass,int64
3,Name,object
4,Sex,object
5,Age,float64
6,SibSp,int64
7,Parch,int64
8,Ticket,object
9,Fare,float64


In [11]:
cols_df['Type'].loc[cols_df['Type'] == 'object'] = "STRING"
cols_df['Type'].loc[cols_df['Type'] == 'int64'] = "INTEGER"
cols_df['Type'].loc[cols_df['Type'] == 'float64'] = "FLOAT"
cols_df['Type'].loc[cols_df['Field'] == 'InvoiceDate'] = "DATE"

cols_df

,Field,Type
0,PassengerId,INTEGER
1,Survived,INTEGER
2,Pclass,INTEGER
3,Name,STRING
4,Sex,STRING
5,Age,FLOAT
6,SibSp,INTEGER
7,Parch,INTEGER
8,Ticket,STRING
9,Fare,FLOAT


In [12]:
schema_dict = {}
cols = []
for index, row in cols_df.iterrows():
  schema_dict[row['Field']]=row['Type']
  cols.append(row['Field'])
schema_dict

{'Age': 'FLOAT',
 'Cabin': 'STRING',
 'Embarked': 'STRING',
 'Fare': 'FLOAT',
 'Name': 'STRING',
 'Parch': 'INTEGER',
 'PassengerId': 'INTEGER',
 'Pclass': 'INTEGER',
 'Sex': 'STRING',
 'SibSp': 'INTEGER',
 'Survived': 'INTEGER',
 'Ticket': 'STRING'}

In [13]:
cols

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [14]:
# creating the schema like apache.beam needs it for writing to BigQuerry
schema = ",".join([row['Field'] + ":" + row['Type'] for index, row in cols_df.iterrows()])
print (schema)   


PassengerId:INTEGER,Survived:INTEGER,Pclass:INTEGER,Name:STRING,Sex:STRING,Age:FLOAT,SibSp:INTEGER,Parch:INTEGER,Ticket:STRING,Fare:FLOAT,Cabin:STRING,Embarked:STRING


## Cleansing and prepping Transform Functions

In [0]:
#using this function to read non ascii text files.

# from https://stackoverflow.com/questions/51935581/apache-beam-gcp-dataflow-encoding-issue
from apache_beam.coders.coders import Coder

class ISOCoder(Coder):
    """A coder used for reading and writing strings as ISO-8859-1."""

    def encode(self, value):
        return value.encode('iso-8859-1')

    def decode(self, value):
        return value.decode('iso-8859-1')

    def is_deterministic(self):
        return True

In [0]:
# splitting date properly and setting it into YYYY-MM-DD format so BigQuerry can read it.
def fix_date(s_date):
  x1 = s_date.split(' ')
  x = x1[0].split("/")
  s_date = "{:04d}-{:02d}-{:02d}".format(2000+int(x[2]),int(x[1]),int(x[0]))

  return s_date

In [0]:
import re
def fix_text(field):
  if field == "":
    return ""
  #cleaing up other fields, could be modified to be done in one regex
  field = field.replace(","," ") 
  field = field.replace('"'," ") 
  
  #removing non standard ascii characeters.
  field = re.sub(r'[^\x00-\x7f]',r'', field)

  return field

In [0]:
def create_row(fields):

    featdict = {}
    for name, value in zip(cols, fields):
        if schema_dict[name] == 'INTEGER':
          if value == '':
            featdict[name] = 0
          else:  
            featdict[name] = int(value)
        if schema_dict[name] == 'FLOAT':
          if value == '':
            featdict[name] = 0
          else:  
            featdict[name] = float(value)
        if schema_dict[name] == 'STRING': 
            featdict[name] = fix_text(value)
        if schema_dict[name] == 'DATE': 
            featdict[name] = fix_date(value)         
    return featdict


# The Magic


In [19]:
ls -al

total 88
drwxr-xr-x 1 root root  4096 Sep  8 19:29 ./
drwxr-xr-x 1 root root  4096 Sep  8 19:13 ../
-rw-r--r-- 1 root root  2497 Sep  8 19:23 adc.json
drwxr-xr-x 1 root root  4096 Sep  8 19:23 .config/
drwxr-xr-x 1 root root  4096 Aug 27 16:17 sample_data/
-rw-r--r-- 1 root root 60302 Sep  8 19:29 titanic.csv


In [21]:
# this is where the magic happens.
pipeline = beam.Pipeline('DirectRunner')
airports = (pipeline
           | beam.io.ReadFromText('gs://arivolibucket/'+fname,skip_header_lines=1,coder=ISOCoder())
           | beam.Map(lambda line: next(csv.reader([line])))
           | beam.Map(lambda line:create_row(line))
           | beam.io.textio.WriteToText('out_temp.csv'))

pipeline.run()

In [22]:
!ls -al

total 268
drwxr-xr-x 1 root root   4096 Sep  8 19:30 .
drwxr-xr-x 1 root root   4096 Sep  8 19:13 ..
-rw-r--r-- 1 root root   2497 Sep  8 19:23 adc.json
drwxr-xr-x 1 root root   4096 Sep  8 19:23 .config
-rw-r--r-- 1 root root 184060 Sep  8 19:30 out_temp.csv-00000-of-00001
drwxr-xr-x 1 root root   4096 Aug 27 16:17 sample_data
-rw-r--r-- 1 root root  60302 Sep  8 19:29 titanic.csv


In [23]:
!head out_temp.csv-00000-of-00001


{'PassengerId': 1, 'Survived': 0, 'Pclass': 3, 'Name': 'Braund  Mr. Owen Harris', 'Sex': 'male', 'Age': 22.0, 'SibSp': 1, 'Parch': 0, 'Ticket': 'A/5 21171', 'Fare': 7.25, 'Cabin': '', 'Embarked': 'S'}
{'PassengerId': 2, 'Survived': 1, 'Pclass': 1, 'Name': 'Cumings  Mrs. John Bradley (Florence Briggs Thayer)', 'Sex': 'female', 'Age': 38.0, 'SibSp': 1, 'Parch': 0, 'Ticket': 'PC 17599', 'Fare': 71.2833, 'Cabin': 'C85', 'Embarked': 'C'}
{'PassengerId': 3, 'Survived': 1, 'Pclass': 3, 'Name': 'Heikkinen  Miss. Laina', 'Sex': 'female', 'Age': 26.0, 'SibSp': 0, 'Parch': 0, 'Ticket': 'STON/O2. 3101282', 'Fare': 7.925, 'Cabin': '', 'Embarked': 'S'}
{'PassengerId': 4, 'Survived': 1, 'Pclass': 1, 'Name': 'Futrelle  Mrs. Jacques Heath (Lily May Peel)', 'Sex': 'female', 'Age': 35.0, 'SibSp': 1, 'Parch': 0, 'Ticket': '113803', 'Fare': 53.1, 'Cabin': 'C123', 'Embarked': 'S'}
{'PassengerId': 5, 'Survived': 0, 'Pclass': 3, 'Name': 'Allen  Mr. William Henry', 'Sex': 'male', 'Age': 35.0, 'SibSp': 0, 'Parc

In [24]:
!tail out_temp.csv-00000-of-00001

{'PassengerId': 882, 'Survived': 0, 'Pclass': 3, 'Name': 'Markun  Mr. Johann', 'Sex': 'male', 'Age': 33.0, 'SibSp': 0, 'Parch': 0, 'Ticket': '349257', 'Fare': 7.8958, 'Cabin': '', 'Embarked': 'S'}
{'PassengerId': 883, 'Survived': 0, 'Pclass': 3, 'Name': 'Dahlberg  Miss. Gerda Ulrika', 'Sex': 'female', 'Age': 22.0, 'SibSp': 0, 'Parch': 0, 'Ticket': '7552', 'Fare': 10.5167, 'Cabin': '', 'Embarked': 'S'}
{'PassengerId': 884, 'Survived': 0, 'Pclass': 2, 'Name': 'Banfield  Mr. Frederick James', 'Sex': 'male', 'Age': 28.0, 'SibSp': 0, 'Parch': 0, 'Ticket': 'C.A./SOTON 34068', 'Fare': 10.5, 'Cabin': '', 'Embarked': 'S'}
{'PassengerId': 885, 'Survived': 0, 'Pclass': 3, 'Name': 'Sutehall  Mr. Henry Jr', 'Sex': 'male', 'Age': 25.0, 'SibSp': 0, 'Parch': 0, 'Ticket': 'SOTON/OQ 392076', 'Fare': 7.05, 'Cabin': '', 'Embarked': 'S'}
{'PassengerId': 886, 'Survived': 0, 'Pclass': 3, 'Name': 'Rice  Mrs. William (Margaret Norton)', 'Sex': 'female', 'Age': 39.0, 'SibSp': 0, 'Parch': 5, 'Ticket': '382652', 

In [28]:
# this is where the magic happens.
bq_table = "titanicproject:load_data.titanic" #change this for your project and dataset

pipeline = beam.Pipeline('DirectRunner')
airports = (pipeline
           | beam.io.ReadFromText('gs://arivolibucket/'+fname,skip_header_lines=1,coder=ISOCoder())
           | beam.Map(lambda line: next(csv.reader([line])))
           | beam.Map(lambda line:create_row(line))
           | 'events:out' >> beam.io.Write(beam.io.BigQuerySink(
                              bq_table, schema=schema,
                              write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE,
                              create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED)))
pipeline.run()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  # This is added back by InteractiveShellApp.init_path()
